In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import filehandling
import os
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [6]:
direc = "/media/data/Data/BallBearing/HIPS/IslandExperiments/New_2,25mm_experiments/75%_15s"
files = filehandling.get_directory_filenames(f"{direc}/*.hdf5")

In [7]:

duties = []
angles = []
for file in files:
    data = pd.read_hdf(file)
    order = np.sum(data.hexatic_order)
    angle = np.angle(order)
    duty = int(file[-8:-5])
    duties.append(duty)
    angles.append(angle)

In [8]:
plt.plot(duties, angles)

In [10]:
from math import pi

In [11]:
duties = []
crystal1 = []
crystal2 = []
for file in files:
    data = pd.read_hdf(file)
    angle = np.angle(data.hexatic_order)
    order1 = np.mean(data.hexatic_order[(-pi/4<angle)&(angle<pi/4)])
    order2 = np.mean(data.hexatic_order[(angle < -pi/4)|(angle > pi/4)])
    duty = int(file[-8:-5])
    duties.append(duty)
    crystal1.append(np.abs(order1))
    crystal2.append(np.abs(order2))

In [12]:
plt.figure()
plt.plot(duties, crystal1, 'c-', label='clusters')
plt.plot(duties, crystal2, 'r--', label='bulk')
plt.xlabel('Duty')
plt.ylabel('$|\psi_6|$')
plt.legend()

## Histograms

In [13]:
all_freqs = {}

In [14]:
rate = direc.split('_')[-1]

angles = np.linspace(-np.pi, np.pi, 100)
freqs = {}
for file in files:
    data = pd.read_hdf(file)
    data['hexatic_abs'] = np.abs(data.hexatic_order)
    thresholded_data = data[data.hexatic_abs > 0.5]
    angle = np.angle(thresholded_data.hexatic_order)
    counts, bins = np.histogram(angle, angles, density=True)
    counts *= len(thresholded_data)/len(data)
    freqs[int(file[-8:-5])] = counts
all_freqs[rate] = freqs

In [15]:
for d, freq in zip(duties, freqs):
    plt.plot(angles[:-1], freq, label=d)
    plt.xlabel('Angle (radians)')
plt.legend()

ValueError: x and y must have same first dimension, but have shapes (99,) and (1,)

In [208]:
duties = 500, 490, 480, 470, 460
for d in duties:
    plt.plot(angles[:-1], freqs[d], label=d)
    plt.xlabel('Angle (radians)')
plt.legend()

In [186]:
for duty in [520, 500, 490, 480, 470, 460]:
    plt.figure()
    for rate in all_freqs.keys():
        plt.plot(angles[:-1], all_freqs[rate][duty], label=rate)
    plt.xlabel('Angle (radians)')
    plt.ylabel('Density')
    plt.title(duty)
    plt.legend()


## Pictures

In [8]:
def plot_duty(d):
    file = f"{direc}/{d}.hdf5"
    data = pd.read_hdf(file)
    frame = data.loc[0]
    plt.figure()
    plt.scatter(frame.x, frame.y, c=np.angle(frame.hexatic_order), cmap='hsv', marker='.')
    plt.title(d)
    plt.axis('equal')

In [9]:
plot_duty(530)
plot_duty(525)
plot_duty(519)
plot_duty(511)
plot_duty(501)
plot_duty(470)

In [217]:

plot_duty(511)
plot_duty(509)
plot_duty(507)
plot_duty(505)
plot_duty(503)

In [71]:
def get_hexatic_counts(data, bins):
    angles = np.angle(data['hexatic_order'])
    counts, _ = np.histogram(angles, bins)

def get_directory_hexatic_counts(direc, bins, threshold=None):
    files = filehandling.get_directory_filenames(f"{direc}/*.hdf5")
    data_files = (pd.read_hdf(f) for f in files)
    duties = [int(os.path.split(f)[1][:3]) for f in files]
    counts_data = {}
    for data, duty in zip(data_files, duties):
        total_count = len(data)
        if threshold is not None:
            data['hexatic_order_abs'] = np.abs(data.hexatic_order)
            data = data.loc[data.hexatic_order_abs>threshold]
        angles = np.angle(data['hexatic_order'])
        if threshold is None:
            counts, _ = np.histogram(angles, bins, density=True)
        else:
            counts, _ = np.histogram(angles, bins)
            counts = counts / total_count

        counts_data[duty] = counts
    return counts_data

In [72]:
parent_direc = "/media/data/Data/BallBearing/HIPS/IslandExperiments/New_1,91mm_experiments"
child_direcs = [
    "85%_1s",
    "85%_5s",
    "85%_15s",
    "85%_30s",
    "85%_300s",
    ]
rates = [s.split('_')[1][:-1] for s in child_direcs]
direcs = [f"{parent_direc}/{c}" for c in child_direcs]

angle_bins = np.linspace(-np.pi, np.pi, 100)
bin_centers = (angle_bins[:-1] + angle_bins[1:])/2

rates_data = {}
for direc, rate in zip(direcs, rates):
    rates_data[rate] = get_directory_hexatic_counts(direc, angle_bins, 0.8)


In [60]:
duties = [500, 490, 480, 470, 460]
for rate, duty_data in rates_data.items():
    plt.figure()
    for duty in duties:
        freqs = duty_data[duty]
        plt.plot(bin_centers, freqs, label=duty)
    plt.title(rate)
    plt.legend()

|## Figure per duty

## Ratios

In [65]:
for rate, duty_data in rates_data.items():
    plt.figure()
    island_data = []
    bulk_data = []
    duties = []
    for duty, freqs in duty_data.items():
        islands = np.sum(freqs[(bin_centers>-0.25*np.pi)*(bin_centers<0.25*np.pi)])
        bulk = np.sum(freqs[(bin_centers<-0.25*np.pi)|(bin_centers>0.25*np.pi)])
        total = islands + bulk
        island_data.append(100*islands/total)
        bulk_data.append(100*bulk/total)
        duties.append(duty)
    plt.plot(duties, bulk_data, label='bulk')
    plt.plot(duties, island_data, label='clusters')
    plt.xlabel('Duty')
    plt.ylabel('%')
    plt.ylim([0, 100])
    plt.legend()
    plt.title(rate)

In [69]:
plt.close('all')

## Fractions

In [77]:
for rate, duty_data in rates_data.items():
    plt.figure()
    island_data = []
    bulk_data = []
    duties = []
    for duty, freqs in duty_data.items():
        islands = np.sum(freqs[(bin_centers>-0.5*np.pi)*(bin_centers<0.5*np.pi)])
        bulk = np.sum(freqs[(bin_centers<-0.5*np.pi)|(bin_centers>0.5*np.pi)])
        total = islands + bulk
        island_data.append(islands)
        bulk_data.append(bulk)
        duties.append(duty)
    plt.plot(duties, bulk_data, label='bulk')
    plt.plot(duties, island_data, label='clusters')
    plt.xlabel('Duty')
    plt.ylabel('%')
    # plt.ylim([0, 100])
    plt.legend()
    plt.title(rate)

In [78]:
plt.close('all')

## Plot examples of different points of the graphs

In [155]:
DIREC = "/media/data/Data/BallBearing/HIPS/IslandExperiments/New_1,91mm_experiments"
SAVEDIREC = "/media/data/Data/BallBearing/HIPS/IslandExperiments/New_1,91mm_experiments/OrderPlots/Scatters"
def plot_image(rate, duty, angle_threshold=np.pi/4):
    data_file = f"{DIREC}/85%_{rate}s/{duty}.hdf5"
    data = pd.read_hdf(data_file)
    data['hex_abs'] = np.abs(data.hexatic_order)
    data['hex_angle'] = np.angle(data.hexatic_order)
    frame = data.loc[0]

    islands = frame.loc[np.abs(frame.hex_angle)<angle_threshold]
    bulk = frame.loc[np.abs(frame.hex_angle)>(np.pi-angle_threshold)]
    nothing = frame.loc[(np.abs(frame.hex_angle)>angle_threshold)*(np.abs(frame.hex_angle)<(np.pi-angle_threshold))]
    plt.figure()
    plt.scatter(nothing.x, nothing.y, c='k', marker='.', s=1)
    plt.scatter(islands.x, islands.y, c='cyan', marker='.', s=1)
    plt.scatter(bulk.x, bulk.y, c='r', marker='.', s=1)
    plt.title(f"{duty} {rate}s")
    plt.axis('equal')
    plt.axis('off')
    plt.savefig(f"{SAVEDIREC}/{rate}_{duty}.png", dpi=600, bbox_inches='tight', pad_inches=0.0)
    plt.show()

In [157]:
plt.close('all')
for rate in [1, 5, 15, 30, 300]:
    for duty in [510, 485, 450]:
        plot_image(rate, duty)

In [158]:
plt.close('all')